In [5]:
# !pip install -r requirements.txt -q

In [1]:
from ultralytics import YOLO
import cv2
import random
import numpy as np
from dotenv import load_dotenv
import os
from inference_sdk import InferenceHTTPClient

In [2]:
from segmentator import *
from detector import Detector

In [3]:
segm = Segmentator(model_path="weights/segmentation.pt")
det = Detector(model_id="hip-joint-detection/2")
head_segm = HeadSegmentator(model_id="femur-head-segmentation/2")

In [4]:
image1 = 'test_data/img1.jpg'
image2 = 'test_data/image_2024-10-04_08-59-24_png.rf.c1dac7f8fb2ed3f0950689b41f877e6f.jpg'
image3 = 'test_data/img3.jpg'
image4 = 'test_data/image_2024-10-04_08-46-53_png.rf.e1eb4001b3f260482098dec04f490213.jpg'
image5 = 'test_data/image_2024-10-04_08-55-22_png.rf.a44311a6ee632be9fb8d0d342e06e64e.jpg'
image6 = 'test_data/image_2024-10-04_08-54-17_png.rf.18717b3145384864f2123e2774fb4371.jpg'
image7 = 'test_data/image_2024-10-04_08-59-24_png.rf.c1dac7f8fb2ed3f0950689b41f877e6f.jpg'
detected = 'test_data/detected.jpg'
segmented = 'test_data/segmented.jpg'

Detect all results

In [11]:
img = image4
image = cv2.imread(img)
det_results = det.detect(img)
head_segm_results = head_segm.segment(img)
segm_results = segm.segment(img)
tang_dict = find_tangent_lines(img, segm_results, det_results)
cm = px_in_cm(det_results)

new_points = tang_dict['new_points']
L_line = tang_dict['left_line_points']
R_line = tang_dict['right_line_points']
L_hole_top_left = tang_dict['left_hole_top_left']
R_hole_top_right = tang_dict['right_hole_top_right']

head_edge_points = get_head_edge_points(head_segm_results)
head_highest_points = get_head_highest_points(head_segm_results, cm) 


L_edge_point, R_edge_point = get_head_edge_points(head_segm_results)

L_highest_point, R_highest_point = head_highest_points[0], head_highest_points[1]
L_right_highest_point = new_points[0]
R_left_highest_point = new_points[1]


0: 640x640 1 big-hole, 2 bone-hollows, 307.4ms
Speed: 2.1ms preprocess, 307.4ms inference, 3.7ms postprocess per image at shape (1, 3, 640, 640)


In [15]:
# segm = Segmentator(model_path="weights/segmentation.pt")
# det = Detector(model_id="hip-joint-detection/2")
# head_segm = HeadSegmentator(model_id="femur-head-segmentation/2")
key_points = get_keypoints(image4, 
                           "weights/segmentation.pt", 
                           "femur-head-segmentation/2",
                           "hip-joint-detection/2")

print(key_points)




0: 640x640 1 big-hole, 2 bone-hollows, 631.8ms
Speed: 5.6ms preprocess, 631.8ms inference, 4.3ms postprocess per image at shape (1, 3, 640, 640)
(array([[[ 35,  90,  35],
        [  1,  53,   0],
        [  0,  18,   0],
        ...,
        [  0,  32,   0],
        [  5,  65,   1],
        [  8,  73,   4]],

       [[ 45,  94,  44],
        [ 81, 127,  80],
        [  0,  17,   0],
        ...,
        [  0,  32,   0],
        [  9,  65,   6],
        [ 29,  90,  26]],

       [[ 34,  74,  32],
        [ 64, 101,  63],
        [  0,  13,   0],
        ...,
        [  0,  17,   0],
        [  0,  20,   0],
        [  0,  33,   0]],

       ...,

       [[ 51,  84,  50],
        [ 64,  94,  65],
        [  0,   8,   0],
        ...,
        [  0,   6,  15],
        [  0,   7,   0],
        [  0,  14,   0]],

       [[ 36,  69,  35],
        [  8,  38,   9],
        [  0,   8,   0],
        ...,
        [  0,   8,   5],
        [  0,  13,   0],
        [ 19,  50,   5]],

       [[ 51,  

In [12]:
L_left_top = (int(L_edge_point[0]), int(L_highest_point[1]))
L_left_bottom = (int(L_edge_point[0]), int(L_hole_top_left[1]))

x0_left, y0_left = L_right_highest_point[0], L_right_highest_point[1]
x1_left, y1_left = L_hole_top_left[0], L_hole_top_left[1]
y_left = L_highest_point[1]

x_intersection_left = x0_left + ((y_left - y0_left) * (x1_left - x0_left)) / (y1_left - y0_left)
L_right_top = (int(x_intersection_left), int(y_left))

L_right_bottom = (int(L_hole_top_left[0]), int(L_hole_top_left[1]))

R_right_top = (int(R_edge_point[0]), int(R_highest_point[1]))
R_right_bottom = (int(R_edge_point[0]), int(R_hole_top_right[1]))

x0_right, y0_right = R_left_highest_point[0], R_left_highest_point[1]
x1_right, y1_right = R_hole_top_right[0], R_hole_top_right[1]
y_right = R_highest_point[1]

x_intersection_right = x0_right + ((y_right - y0_right) * (x1_right - x0_right)) / (y1_right - y0_right)
R_left_top = (int(x_intersection_right), int(y_right))

R_left_bottom = (int(R_hole_top_right[0]), int(R_hole_top_right[1]))


In [13]:
cv2.circle(image, L_left_top, 5, (0, 0, 255), -1)
cv2.circle(image, L_right_top, 5, (0, 255, 0), -1)
cv2.circle(image, L_left_bottom, 5, (0, 0, 255), -1)
cv2.circle(image, L_right_bottom, 5, (0, 0, 255), -1)
cv2.circle(image, R_left_top, 5, (0, 255, 0), -1)
cv2.circle(image, R_right_top, 5, (0, 0, 255), -1)
cv2.circle(image, R_left_bottom, 5, (0, 0, 255), -1)
cv2.circle(image, R_right_bottom, 5, (0, 0, 255), -1)
cv2.imwrite(detected, image)

True

In [9]:
cv2.imwrite('test_data/tangent_lines.jpg', image)

True